In [1]:
!pip install /kaggle/input/mypackages/mypackages/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/mypackages/mypackages/ultralytics-8.0.124-py3-none-any.whl
!pip install /kaggle/input/mypackages/mypackages/ensemble_boxes-1.0.9-py3-none-any.whl

Processing /kaggle/input/mypackages/mypackages/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/mypackages/mypackages/ultralytics-8.0.124-py3-none-any.whl
Processing /kaggle/input/mypackages/mypackages/ensemble_boxes-1.0.9-py3-none-any.whl


In [2]:
import os
import sys

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [3]:
from numba import cuda
import gc
import torch
import time
import cv2
from ultralytics import YOLO
from PIL import Image
import random
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import albumentations
import timm
from collections import OrderedDict
from warnings import filterwarnings
filterwarnings("ignore")
device = torch.device('cuda')
torch.no_grad()

from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import seaborn as sns

In [4]:
# 设置随机种子，以便实验复现

def set_seed(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed()

# 1. dcm to png

In [5]:
# from https://www.kaggle.com/code/xhlulu/siim-covid-19-convert-to-jpg-256px?scriptVersionId=63196459
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)

    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [6]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [7]:
image_id = []
dim0 = []
dim1 = []
splits = []
study_id = []

split = 'test'

save_dir = f'/kaggle/working/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)
    
for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
    for file in filenames:
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size=512)  
        im.save(os.path.join(save_dir, file.replace('dcm', 'png')))

        image_id.append(file.replace('.dcm', ''))
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])
        splits.append(split)
        study_id.append(dirname.split('/')[-2])

0it [00:00, ?it/s]

In [8]:
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits, 'study_id':study_id})
meta = meta[meta['split'] == 'test']
meta['image_id'] = meta['image_id'] + '_image'
meta['study_id'] = meta['study_id'] + '_study'
meta.columns = ['id', 'dim0', 'dim1', 'split', 'study_id']
meta

,id,dim0,dim1,split,study_id
0,a29c5a68b07b_image,2320,2828,test,2fb11712bc93_study
1,9850b5470fd6_image,2330,2382,test,19c66935e737_study
2,8d6dea06a032_image,2422,3344,test,2fc50bf199cd_study
3,dfc5c09a50bc_image,1140,1387,test,97c5d6eb413d_study
4,7230234e120a_image,2318,2383,test,593c3f815635_study
...,...,...,...,...,...
1258,a43200bd5ceb_image,2336,2836,test,b12d2e5c39b7_study
1259,37bf83df1b86_image,3480,4240,test,6484393291ea_study
1260,b30d2aef985f_image,3480,4240,test,6484393291ea_study
1261,a37a362df0ac_image,3480,4240,test,531aa20ff7c3_study


In [9]:
all_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
all_df['id_last_str'] = all_df["id"].str[-1]

study_len = all_df[all_df['id_last_str'] == 'y'].shape[0]

all_df

,id,PredictionString,id_last_str
0,00188a671292_study,negative 1 0 0 1 1,y
1,004bd59708be_study,negative 1 0 0 1 1,y
2,00508faccd39_study,negative 1 0 0 1 1,y
3,006486aa80b2_study,negative 1 0 0 1 1,y
4,00655178fdfc_study,negative 1 0 0 1 1,y
...,...,...,...
2472,46719b856de1_image,none 1 0 0 1 1,e
2473,31c07523a69a_image,none 1 0 0 1 1,e
2474,f77d7d1aebab_image,none 1 0 0 1 1,e
2475,ccc5b63ca96d_image,none 1 0 0 1 1,e


# 2. 2-class Predict on Image Level

In [10]:
image_sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
image_sub_df = image_sub_df[study_len:].reset_index(drop=True)

image_sub_df["file_name"] = image_sub_df["id"].str.replace('_image', '') + ".png"
image_sub_df['none'] = 0
image_sub_df

,id,PredictionString,file_name,none
0,557a70442928_image,none 1 0 0 1 1,557a70442928.png,0
1,36141cda67ad_image,none 1 0 0 1 1,36141cda67ad.png,0
2,2413a23a5477_image,none 1 0 0 1 1,2413a23a5477.png,0
3,c263b1e9aa64_image,none 1 0 0 1 1,c263b1e9aa64.png,0
4,4fe0444d7fc5_image,none 1 0 0 1 1,4fe0444d7fc5.png,0
...,...,...,...,...
1258,46719b856de1_image,none 1 0 0 1 1,46719b856de1.png,0
1259,31c07523a69a_image,none 1 0 0 1 1,31c07523a69a.png,0
1260,f77d7d1aebab_image,none 1 0 0 1 1,f77d7d1aebab.png,0
1261,ccc5b63ca96d_image,none 1 0 0 1 1,ccc5b63ca96d.png,0


## 2-class Predict

In [11]:
model_path = '/kaggle/input/yolov8/yolov8class2.pt'
model = YOLO(model_path)
model.predict(source=f'/kaggle/working/tmp/{split}',name='2class',imgsz=512, save_txt=True, save_conf=True)

torch.cuda.empty_cache()
gc.collect()



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1263 /kaggle/working/tmp/test/0026720152f5.png: 512x512 opacity 1.00, none 0.00, 15.1ms
image 2/1263 /kaggle/working/tmp/test/00fc8fc35dc1.png: 512x512 opacity 1.00, none 0.00, 15.1ms
image 3/1263 /kaggle/working/tmp/test/01c3512eebc3.png: 512x512 opacity 0.78, none 0.22, 15.0ms
image 4/1263 /kaggle/working/tmp/test/01f948f8e544.png: 512x512 none 1.00, opacity 0.00, 15.0ms
image 5/1263 /kaggle/working/tmp/test/022146012034.png: 512x512 none 1.00, 

0

In [12]:
predictions = []

for i in range(len(image_sub_df)):
    image_id = image_sub_df.loc[i,'id']
    save_dir = '/kaggle/working/runs/classify/2class/labels/{}.txt'.format(image_id.replace('_image', ''))
    f = open(save_dir, "r")
    data = np.array(f.read().replace('\n', ' ').strip().split(' '))
    if data[1] == 'none':
        predictions.append(data[0])
    else:
        predictions.append(data[2])

shutil.rmtree('/kaggle/working/runs')

image_sub_df['none'] = predictions
image_sub_df

,id,PredictionString,file_name,none
0,557a70442928_image,none 1 0 0 1 1,557a70442928.png,0.98
1,36141cda67ad_image,none 1 0 0 1 1,36141cda67ad.png,0.00
2,2413a23a5477_image,none 1 0 0 1 1,2413a23a5477.png,0.00
3,c263b1e9aa64_image,none 1 0 0 1 1,c263b1e9aa64.png,0.18
4,4fe0444d7fc5_image,none 1 0 0 1 1,4fe0444d7fc5.png,0.01
...,...,...,...,...
1258,46719b856de1_image,none 1 0 0 1 1,46719b856de1.png,0.92
1259,31c07523a69a_image,none 1 0 0 1 1,31c07523a69a.png,0.00
1260,f77d7d1aebab_image,none 1 0 0 1 1,f77d7d1aebab.png,0.87
1261,ccc5b63ca96d_image,none 1 0 0 1 1,ccc5b63ca96d.png,0.00


In [13]:
image_sub_df = image_sub_df.merge(meta,how="left",on='id')
image_sub_df

,id,PredictionString,file_name,none,dim0,dim1,split,study_id
0,557a70442928_image,none 1 0 0 1 1,557a70442928.png,0.98,3320,3408,test,795051254905_study
1,36141cda67ad_image,none 1 0 0 1 1,36141cda67ad.png,0.00,3480,4240,test,0d476d070d71_study
2,2413a23a5477_image,none 1 0 0 1 1,2413a23a5477.png,0.00,2320,2832,test,4996bf5117c7_study
3,c263b1e9aa64_image,none 1 0 0 1 1,c263b1e9aa64.png,0.18,2801,2802,test,8b273337a684_study
4,4fe0444d7fc5_image,none 1 0 0 1 1,4fe0444d7fc5.png,0.01,2392,3014,test,aafc8126d5a1_study
...,...,...,...,...,...,...,...,...
1258,46719b856de1_image,none 1 0 0 1 1,46719b856de1.png,0.92,2648,2867,test,ee860264dd8c_study
1259,31c07523a69a_image,none 1 0 0 1 1,31c07523a69a.png,0.00,3488,4256,test,81c860c6efe8_study
1260,f77d7d1aebab_image,none 1 0 0 1 1,f77d7d1aebab.png,0.87,3052,3012,test,e6e02ec8aff5_study
1261,ccc5b63ca96d_image,none 1 0 0 1 1,ccc5b63ca96d.png,0.00,2520,3032,test,b93bfa119338_study


## Yolo Predict

In [14]:
yolo_df = all_df[study_len:].reset_index(drop=True)
yolo_df = pd.merge(yolo_df, meta, on='id', how='left')
yolo_df = yolo_df.drop(['PredictionString'], axis=1)
yolo_df

,id,id_last_str,dim0,dim1,split,study_id
0,557a70442928_image,e,3320,3408,test,795051254905_study
1,36141cda67ad_image,e,3480,4240,test,0d476d070d71_study
2,2413a23a5477_image,e,2320,2832,test,4996bf5117c7_study
3,c263b1e9aa64_image,e,2801,2802,test,8b273337a684_study
4,4fe0444d7fc5_image,e,2392,3014,test,aafc8126d5a1_study
...,...,...,...,...,...,...
1258,46719b856de1_image,e,2648,2867,test,ee860264dd8c_study
1259,31c07523a69a_image,e,3488,4256,test,81c860c6efe8_study
1260,f77d7d1aebab_image,e,3052,3012,test,e6e02ec8aff5_study
1261,ccc5b63ca96d_image,e,2520,3032,test,b93bfa119338_study


In [15]:
for fold in range(5):
    model_path = f'/kaggle/input/yolov8/folds/fold{str(fold)}_best.pt'
    model = YOLO(model_path)
    model.predict(source=f'/kaggle/working/tmp/{split}',name=f'fold{str(fold)}',imgsz=512, augment=True, conf=0.001, iou=0.5, save_txt=True, save_conf=True)
    torch.cuda.empty_cache()
    gc.collect()

model_path = '/kaggle/input/yolov8/yolov8.pt'
model = YOLO(model_path)
result = model.predict(source=f'/kaggle/working/tmp/{split}',name='best',imgsz=512, augment=True, conf=0.001, iou=0.5, save_txt=True, save_conf=True)
torch.cuda.empty_cache()
gc.collect()



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1263 /kaggle/working/tmp/test/0026720152f5.png: 512x512 32 opacitys, 146.7ms
image 2/1263 /kaggle/working/tmp/test/00fc8fc35dc1.png: 512x512 12 opacitys, 66.1ms
image 3/1263 /kaggle/working/tmp/test/01c3512eebc3.png: 512x512 22 opacitys, 65.7ms
image 4/1263 /kaggle/working/tmp/test/01f948f8e544.png: 512x512 22 opacitys, 65.7ms
image 5/1263 /kaggle/working/tmp/test/022146012034.png: 512x512 11 opacitys, 65.9ms
image 6/1263 /kaggle/working/tmp/test/

4751

In [16]:
def yolo2voc(bboxes):
    bboxes = bboxes.copy().astype(float)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes

def yolo2voc1(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float)
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    return bboxes

In [17]:
from ensemble_boxes import weighted_boxes_fusion
image_ids = []
PredictionStrings = []

for i in range(len(yolo_df)):
    is_exist1 = True
    is_exist2 = True
    
    iou_thr = 0.5
    skip_box_thr = 0.0001
    image_id = yolo_df.loc[i,'id']
    w, h = yolo_df.loc[yolo_df.id==image_id,['dim1', 'dim0']].values[0]

    weights = [1, 1, 1, 1, 1]

    boxes_list = []
    scores_list = []
    labels_list = []

    yolo_save_dirs = [f'/kaggle/working/runs/detect/fold{fold}/labels' for fold in range(5)]
    
    for save_dir in yolo_save_dirs:
        pth = save_dir + '/{}.txt'
        file_path = pth.format(image_id.replace('_image', ''))
        if os.path.exists(file_path):
            f = open(file_path, 'r')
            data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
            data = data[:, [0, 5, 1, 2, 3, 4]]
            data[:,2:] = yolo2voc(data[:, 2:])
            boxes_list.append(data[:,2:].tolist())
            scores_list.append(data[:,1].tolist())
            labels_list.append(data[:,0].tolist())

    if len(boxes_list)>1:
        if len(boxes_list)==2:
            weights = [1, 1]
        elif len(boxes_list)==3:
            weights = [1, 1, 1]
        elif len(boxes_list)==4:
            weights = [1, 1, 1, 1]
        else:
            weights = [1, 1, 2, 1, 1]
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        data1 = np.zeros((boxes.shape[0],6))
        data1[:,0] = labels
        data1[:,1] = scores
        data1[:,2:] = boxes
    elif len(boxes_list)==1:
        data1 = data
    else:
        is_exist1 = False


    boxes_list = []
    scores_list = []
    labels_list = []

    file_path = '/kaggle/working/runs/detect/best/labels/{}.txt'.format(image_id.replace('_image', ''))
    if os.path.exists(file_path):
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        data[:,2:] = yolo2voc(data[:, 2:])

    weights = [1, 6]
    if os.path.exists(file_path) and is_exist1:
        boxes_list.append(data[:,2:].tolist())
        scores_list.append(data[:,1].tolist())
        labels_list.append(data[:,0].tolist())
        boxes_list.append(data1[:,2:].tolist())
        scores_list.append(data1[:,1].tolist())
        labels_list.append(data1[:,0].tolist())
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        data2 = np.zeros((boxes.shape[0],6))
        data2[:,0] = labels
        data2[:,1] = scores
        data2[:,2:] = boxes
    elif os.path.exists(file_path) or is_exist1:
        data2 = data
    else:
        is_exist2 = False
    if is_exist1==False and is_exist2==False:
        continue
            
    bboxes = list(np.round(np.concatenate((data2[:, :2], np.round(yolo2voc1(h, w, data2[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

shutil.rmtree('/kaggle/working/runs')

yolo_predict_df = pd.DataFrame({'id':image_ids, 'PredictionString':PredictionStrings})
yolo_predict_df

,id,PredictionString
0,557a70442928_image,1 0.13438635213 2303 1793 2960 2211 1 0.016745...
1,36141cda67ad_image,1 0.095072984695 3145 1447 3719 2422 1 0.04068...
2,2413a23a5477_image,1 0.298962082182 1657 780 2421 1718 1 0.202971...
3,c263b1e9aa64_image,1 0.288801363536 408 989 1119 1918 1 0.2477065...
4,4fe0444d7fc5_image,1 0.177460977009 1989 870 2865 1598 1 0.115512...
...,...,...
1258,46719b856de1_image,1 0.028545443501 1665 1003 2406 1829 1 0.02500...
1259,31c07523a69a_image,1 0.530465875353 2769 732 4074 2811 1 0.314306...
1260,f77d7d1aebab_image,1 0.015601596662 1867 1538 2432 2092 1 0.01106...
1261,ccc5b63ca96d_image,1 0.002934679921 1937 711 2944 2313 1 0.001166...


In [18]:
yolo_df = pd.merge(yolo_df, yolo_predict_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
yolo_df = yolo_df[['id', 'PredictionString']]
yolo_df

,id,PredictionString
0,557a70442928_image,1 0.13438635213 2303 1793 2960 2211 1 0.016745...
1,36141cda67ad_image,1 0.095072984695 3145 1447 3719 2422 1 0.04068...
2,2413a23a5477_image,1 0.298962082182 1657 780 2421 1718 1 0.202971...
3,c263b1e9aa64_image,1 0.288801363536 408 989 1119 1918 1 0.2477065...
4,4fe0444d7fc5_image,1 0.177460977009 1989 870 2865 1598 1 0.115512...
...,...,...
1258,46719b856de1_image,1 0.028545443501 1665 1003 2406 1829 1 0.02500...
1259,31c07523a69a_image,1 0.530465875353 2769 732 4074 2811 1 0.314306...
1260,f77d7d1aebab_image,1 0.015601596662 1867 1538 2432 2092 1 0.01106...
1261,ccc5b63ca96d_image,1 0.002934679921 1937 711 2944 2313 1 0.001166...


In [19]:
for i in range(yolo_df.shape[0]):
    if yolo_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = yolo_df.loc[i,'PredictionString'].split()
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    yolo_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)

yolo_df['none'] = image_sub_df['none'] 
for i in range(yolo_df.shape[0]):
    if yolo_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        yolo_df.loc[i,'PredictionString'] = yolo_df.loc[i,'PredictionString'] + ' none ' + str(yolo_df.loc[i,'none']) + ' 0 0 1 1'

yolo_df = yolo_df[['id', 'PredictionString']] 
# yolo_df.to_csv('yolo_df.csv', index=False)
image_df = yolo_df
image_df

,id,PredictionString
0,557a70442928_image,opacity 0.13438635213 2303 1793 2960 2211 opac...
1,36141cda67ad_image,opacity 0.095072984695 3145 1447 3719 2422 opa...
2,2413a23a5477_image,opacity 0.298962082182 1657 780 2421 1718 opac...
3,c263b1e9aa64_image,opacity 0.288801363536 408 989 1119 1918 opaci...
4,4fe0444d7fc5_image,opacity 0.177460977009 1989 870 2865 1598 opac...
...,...,...
1258,46719b856de1_image,opacity 0.028545443501 1665 1003 2406 1829 opa...
1259,31c07523a69a_image,opacity 0.530465875353 2769 732 4074 2811 opac...
1260,f77d7d1aebab_image,opacity 0.015601596662 1867 1538 2432 2092 opa...
1261,ccc5b63ca96d_image,opacity 0.002934679921 1937 711 2944 2313 opac...


# 3. 4-class Predict on Study Level

In [20]:
study_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
study_df = study_df[:study_len]
study_df

,id,PredictionString
0,00188a671292_study,negative 1 0 0 1 1
1,004bd59708be_study,negative 1 0 0 1 1
2,00508faccd39_study,negative 1 0 0 1 1
3,006486aa80b2_study,negative 1 0 0 1 1
4,00655178fdfc_study,negative 1 0 0 1 1
...,...,...
1209,ff1ba0e9aaf0_study,negative 1 0 0 1 1
1210,ff2cc4de58c5_study,negative 1 0 0 1 1
1211,ff2f0a744930_study,negative 1 0 0 1 1
1212,ff88940dce8b_study,negative 1 0 0 1 1


In [21]:
meta_study = meta.drop_duplicates(['study_id'], keep='last')
meta_study.columns = ['image_id', 'dim0', 'dim1', 'split', 'id']
meta_study

,image_id,dim0,dim1,split,id
0,a29c5a68b07b_image,2320,2828,test,2fb11712bc93_study
1,9850b5470fd6_image,2330,2382,test,19c66935e737_study
2,8d6dea06a032_image,2422,3344,test,2fc50bf199cd_study
3,dfc5c09a50bc_image,1140,1387,test,97c5d6eb413d_study
4,7230234e120a_image,2318,2383,test,593c3f815635_study
...,...,...,...,...,...
1257,cede944b70a5_image,2539,3050,test,57e2c6e368f2_study
1258,a43200bd5ceb_image,2336,2836,test,b12d2e5c39b7_study
1260,b30d2aef985f_image,3480,4240,test,6484393291ea_study
1261,a37a362df0ac_image,3480,4240,test,531aa20ff7c3_study


In [22]:
study_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
study_df = study_df[:study_len]

study_df = pd.merge(study_df, meta_study, on='id', how='left')

study_df['file_name'] = study_df['image_id'].str.replace('_image', '') + ".png"
study_df = study_df[['id', 'PredictionString', 'file_name']]

study_df['negative'] = 0
study_df['typical'] = 0
study_df['indeterminate'] = 0
study_df['atypical'] = 0

study_df

,id,PredictionString,file_name,negative,typical,indeterminate,atypical
0,00188a671292_study,negative 1 0 0 1 1,3dcdfc352a06.png,0,0,0,0
1,004bd59708be_study,negative 1 0 0 1 1,c39146cbda47.png,0,0,0,0
2,00508faccd39_study,negative 1 0 0 1 1,951211f8e1bb.png,0,0,0,0
3,006486aa80b2_study,negative 1 0 0 1 1,5e0e7acd9c7d.png,0,0,0,0
4,00655178fdfc_study,negative 1 0 0 1 1,5b8ee5baa1d5.png,0,0,0,0
...,...,...,...,...,...,...,...
1209,ff1ba0e9aaf0_study,negative 1 0 0 1 1,064b37b01cd2.png,0,0,0,0
1210,ff2cc4de58c5_study,negative 1 0 0 1 1,02eceb0fc405.png,0,0,0,0
1211,ff2f0a744930_study,negative 1 0 0 1 1,16a0d37747d7.png,0,0,0,0
1212,ff88940dce8b_study,negative 1 0 0 1 1,aa55fc6c616c.png,0,0,0,0


In [23]:
from ultralytics import YOLO
from PIL import Image
import cv2

model_path = '/kaggle/input/yolov8/yolov8class4.pt'
model = YOLO(model_path)
model.predict(source=f'/kaggle/working/tmp/{split}',name='4class',imgsz=512, save_txt=True, save_conf=True)
torch.cuda.empty_cache()
gc.collect()



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1263 /kaggle/working/tmp/test/0026720152f5.png: 512x512 Typical Appearance 0.86, Negative for Pneumonia 0.13, Indeterminate Appearance 0.00, Atypical Appearance 0.00, 15.1ms
image 2/1263 /kaggle/working/tmp/test/00fc8fc35dc1.png: 512x512 Indeterminate Appearance 0.52, Typical Appearance 0.48, Negative for Pneumonia 0.00, Atypical Appearance 0.00, 15.2ms
image 3/1263 /kaggle/working/tmp/test/01c3512eebc3.png: 512x512 Negative for Pneumonia 0.74, Ty

4751

In [24]:
negative = []
typical = []
indeterminate = []
atypical = []

for i in range(len(study_df)):
    file_name = study_df.loc[i,'file_name']
    save_dir = '/kaggle/working/runs/classify/4class/labels/{}.txt'.format(file_name.replace('.png', ''))
    f = open(save_dir, "r")
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).tolist()
    idx = data.index('Negative')
    negative.append(data[idx-1])
    idx = data.index('Typical')
    typical.append(data[idx-1])
    idx = data.index('Indeterminate')
    indeterminate.append(data[idx-1])
    idx = data.index('Atypical')
    atypical.append(data[idx-1])

shutil.rmtree('/kaggle/working/runs')

study_df['negative'] = negative
study_df['typical'] = typical
study_df['indeterminate'] = indeterminate
study_df['atypical'] = atypical
study_df

,id,PredictionString,file_name,negative,typical,indeterminate,atypical
0,00188a671292_study,negative 1 0 0 1 1,3dcdfc352a06.png,1.00,0.00,0.00,0.00
1,004bd59708be_study,negative 1 0 0 1 1,c39146cbda47.png,0.00,1.00,0.00,0.00
2,00508faccd39_study,negative 1 0 0 1 1,951211f8e1bb.png,0.02,0.01,0.97,0.00
3,006486aa80b2_study,negative 1 0 0 1 1,5e0e7acd9c7d.png,0.00,1.00,0.00,0.00
4,00655178fdfc_study,negative 1 0 0 1 1,5b8ee5baa1d5.png,0.00,0.00,1.00,0.00
...,...,...,...,...,...,...,...
1209,ff1ba0e9aaf0_study,negative 1 0 0 1 1,064b37b01cd2.png,0.00,0.99,0.00,0.00
1210,ff2cc4de58c5_study,negative 1 0 0 1 1,02eceb0fc405.png,0.95,0.05,0.00,0.00
1211,ff2f0a744930_study,negative 1 0 0 1 1,16a0d37747d7.png,0.00,1.00,0.00,0.00
1212,ff88940dce8b_study,negative 1 0 0 1 1,aa55fc6c616c.png,1.00,0.00,0.00,0.00


In [25]:
study_df = study_df.drop("file_name",axis=1)
study_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
all_df = pd.merge(all_df, study_df, on = 'id', how = 'left')
all_df

,id,PredictionString,id_last_str,PredictionString1,negative,typical,indeterminate,atypical
0,00188a671292_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,1.00,0.00,0.00,0.00
1,004bd59708be_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.00,1.00,0.00,0.00
2,00508faccd39_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.02,0.01,0.97,0.00
3,006486aa80b2_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.00,1.00,0.00,0.00
4,00655178fdfc_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.00,0.00,1.00,0.00
...,...,...,...,...,...,...,...,...
2472,46719b856de1_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN
2473,31c07523a69a_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN
2474,f77d7d1aebab_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN
2475,ccc5b63ca96d_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN


In [26]:
for i in range(study_len):
    negative = all_df.loc[i,'negative']
    typical = all_df.loc[i,'typical']
    indeterminate = all_df.loc[i,'indeterminate']
    atypical = all_df.loc[i,'atypical']
    all_df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
    
study_df = all_df[['id', 'PredictionString']][:study_len]
# study_df.to_csv('study_df.csv', index=False)
study_df

,id,PredictionString
0,00188a671292_study,negative 1.00 0 0 1 1 typical 0.00 0 0 1 1 ind...
1,004bd59708be_study,negative 0.00 0 0 1 1 typical 1.00 0 0 1 1 ind...
2,00508faccd39_study,negative 0.02 0 0 1 1 typical 0.01 0 0 1 1 ind...
3,006486aa80b2_study,negative 0.00 0 0 1 1 typical 1.00 0 0 1 1 ind...
4,00655178fdfc_study,negative 0.00 0 0 1 1 typical 0.00 0 0 1 1 ind...
...,...,...
1209,ff1ba0e9aaf0_study,negative 0.00 0 0 1 1 typical 0.99 0 0 1 1 ind...
1210,ff2cc4de58c5_study,negative 0.95 0 0 1 1 typical 0.05 0 0 1 1 ind...
1211,ff2f0a744930_study,negative 0.00 0 0 1 1 typical 1.00 0 0 1 1 ind...
1212,ff88940dce8b_study,negative 1.00 0 0 1 1 typical 0.00 0 0 1 1 ind...


# 4. Merge Study and Image

In [27]:
image_df = meta.merge(yolo_df,on="id")
image_df

,id,dim0,dim1,split,study_id,PredictionString
0,a29c5a68b07b_image,2320,2828,test,2fb11712bc93_study,opacity 0.453987121582 653 531 1488 1907 opaci...
1,9850b5470fd6_image,2330,2382,test,19c66935e737_study,opacity 0.30024102756 263 680 1017 1906 opacit...
2,8d6dea06a032_image,2422,3344,test,2fc50bf199cd_study,opacity 0.119990297726 296 1191 962 2197 opaci...
3,dfc5c09a50bc_image,1140,1387,test,97c5d6eb413d_study,opacity 0.064530415194 879 446 1141 759 opacit...
4,7230234e120a_image,2318,2383,test,593c3f815635_study,opacity 0.48258297784 1443 1153 2147 1887 opac...
...,...,...,...,...,...,...
1258,a43200bd5ceb_image,2336,2836,test,b12d2e5c39b7_study,opacity 0.225861719676 1528 315 2221 1676 opac...
1259,37bf83df1b86_image,3480,4240,test,6484393291ea_study,opacity 0.266430905887 2375 453 3500 2069 opac...
1260,b30d2aef985f_image,3480,4240,test,6484393291ea_study,opacity 0.365218060357 739 473 1897 2090 opaci...
1261,a37a362df0ac_image,3480,4240,test,531aa20ff7c3_study,opacity 0.034480933632 612 1145 1769 2177 opac...


In [28]:
from ensemble_boxes import *

weights = [3, 1]
iou_thr = 0.6
skip_box_thr = 0.0001
image_output_list = []

for index, id_, dim0, dim1, split, study_id, yolo5_boxes in image_df.itertuples():
    yolo5_boxes_list = yolo5_boxes.split(" ")
    assert len(yolo5_boxes_list) % 6 == 0
    none_list = yolo5_boxes_list[-6:]
    yolo5_boxes_list = yolo5_boxes_list[:-6]
    boxes_list = [[],[]]
    scores_list = [[],[]]
    for i in range(len(yolo5_boxes_list)):
        if i % 6 == 1:
            scores_list[0].append(float(yolo5_boxes_list[i]))
        if i % 6 == 2:
            boxes_list[0].append([float(yolo5_boxes_list[i])/dim1,
                                  float(yolo5_boxes_list[i+1])/dim0,
                                  float(yolo5_boxes_list[i+2])/dim1,
                                  float(yolo5_boxes_list[i+3])/dim0])

            
    labels_list = [[0]*len(scores_list[0]), [0]*len(scores_list[1])]        
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    output_str = ""
    for box, score in zip(boxes, scores): 
        output_str += "opacity "
        output_str += f"{score} "
        for b_i in range(4):
            if b_i in [0,2]:
                output_str += f"{int(round(box[b_i]*dim1,0))} "
            elif b_i in [1,3]:
                output_str += f"{int(round(box[b_i]*dim0,0))} "

    output_str += " ".join(none_list)
    image_output_list.append(output_str)

    
image_df["PredictionString"] = image_output_list
image_df = image_df[["id", "PredictionString"]]
image_df

,id,PredictionString
0,a29c5a68b07b_image,opacity 0.3404903411865 653 531 1488 1907 opac...
1,9850b5470fd6_image,opacity 0.22518077066999997 263 680 1017 1906 ...
2,8d6dea06a032_image,opacity 0.0899927232945 296 1191 962 2197 opac...
3,dfc5c09a50bc_image,opacity 0.04839781139550001 879 446 1141 759 o...
4,7230234e120a_image,opacity 0.36193723338 1443 1153 2147 1887 opac...
...,...,...
1258,a43200bd5ceb_image,opacity 0.169396289757 1528 315 2221 1676 opac...
1259,37bf83df1b86_image,opacity 0.19982317941525002 2375 453 3500 2069...
1260,b30d2aef985f_image,opacity 0.27391354526775 739 473 1897 2090 opa...
1261,a37a362df0ac_image,opacity 0.025860700224 612 1145 1769 2177 opac...


In [29]:
submission_df = study_df.append(image_df).reset_index(drop=True)
submission_df.to_csv('/kaggle/working/submission.csv',index = False)
submission_df

,id,PredictionString
0,00188a671292_study,negative 1.00 0 0 1 1 typical 0.00 0 0 1 1 ind...
1,004bd59708be_study,negative 0.00 0 0 1 1 typical 1.00 0 0 1 1 ind...
2,00508faccd39_study,negative 0.02 0 0 1 1 typical 0.01 0 0 1 1 ind...
3,006486aa80b2_study,negative 0.00 0 0 1 1 typical 1.00 0 0 1 1 ind...
4,00655178fdfc_study,negative 0.00 0 0 1 1 typical 0.00 0 0 1 1 ind...
...,...,...
2472,a43200bd5ceb_image,opacity 0.169396289757 1528 315 2221 1676 opac...
2473,37bf83df1b86_image,opacity 0.19982317941525002 2375 453 3500 2069...
2474,b30d2aef985f_image,opacity 0.27391354526775 739 473 1897 2090 opa...
2475,a37a362df0ac_image,opacity 0.025860700224 612 1145 1769 2177 opac...
